In [33]:
!pip install cleantext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 13.6 MB/s eta 0:00:00


In [34]:
import cleantext
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

In [132]:
data = pd.read_csv("530finalprojectdata/530datasetV2_train_sample.csv")

In [133]:
data

,Unnamed: 0.1,Unnamed: 0,Name,Song Title,Publication Date,Gender,Genre,Lyrics
0,0,78618,Depeche Mode,And Then...,2003-03-03,Male,NaN,Let&apos;s take a map of the world<br>Tear it ...
1,1,10575,Anathema,Your Possible Pasts,1998-08-11,Mixed,NaN,"They flutter behind you, your possible pasts<b..."
2,2,330145,They Might Be Giants,There,2013-03-10,Male,NaN,There!<br>Near the borderline<br>Right between...
3,3,267149,Showaddywaddy,Multiplication,2009-10-12,Male,NaN,"When you see a gentleman bee, <br>&apos;Round ..."
4,4,53230,Caetano Veloso,Jasper,2015-04-02,Male,NaN,Time is as weak as water<br>I&apos;m kneeling ...
...,...,...,...,...,...,...,...,...
996,996,300521,Test Icicles,What's Your Damage,2006-01-15,Male,NaN,Did you ever think about the words<br>Coming o...
997,997,160344,Josh Groban,Below The Line,2013-02-01,Male,NaN,Big world<br>Please do not swallow us up<br>Pl...
998,998,221920,Nicolai Dunger,"All The Love, Days And Tears",2001-01-30,Male,NaN,"All the love, all the love we shared<br>is all..."
999,999,79107,Day Of Fire,Through The Fire,2001-09-25,Male,NaN,"Walk in the flame again,<br>I&apos;ll be there..."


In [134]:
data['Lyrics'] = data.Lyrics.apply(lambda x : x.replace("&apos;","'"))

In [135]:
data = data.drop('Unnamed: 0.1',axis=1).rename({"Unnamed: 0": "id"},axis=1)

In [136]:
data['Lyrics'] = data.Lyrics.apply(lambda x: x.lower().split("<br>"))
data = data.explode('Lyrics')
data = data[data.Lyrics != ''].reset_index(drop=True)

In [137]:
data

,id,Name,Song Title,Publication Date,Gender,Genre,Lyrics
0,78618,Depeche Mode,And Then...,2003-03-03,Male,NaN,let's take a map of the world
1,78618,Depeche Mode,And Then...,2003-03-03,Male,NaN,tear it into pieces
2,78618,Depeche Mode,And Then...,2003-03-03,Male,NaN,all of the boys and the girls
3,78618,Depeche Mode,And Then...,2003-03-03,Male,NaN,will see how easy it is
4,78618,Depeche Mode,And Then...,2003-03-03,Male,NaN,to put it all down and start again
...,...,...,...,...,...,...,...
34841,58761,Client,Lights Go Out,2007-03-22,Female,NaN,"i stand guilty, loving you again"
34842,58761,Client,Lights Go Out,2007-03-22,Female,NaN,"no mercy, no pleasure, no sin"
34843,58761,Client,Lights Go Out,2007-03-22,Female,NaN,"i stand guilty, loving you again"
34844,58761,Client,Lights Go Out,2007-03-22,Female,NaN,"no mercy, no pleasure, no sin"


In [98]:
import json
f = open("530finalprojectdata/gendered_words.json")
gender_dict = json.load(f)

In [100]:
gender_dict = pd.DataFrame(gender_dict).set_index('word').to_dict()['gender']

In [138]:
data['words'] = data.Lyrics.apply(lambda x: len([y for y in x.split() if y in gender_dict]))

In [139]:
data['male'] = data.Lyrics.apply(lambda x: len([y for y in x.split() if y in gender_dict and gender_dict[y] == 'm']))

In [140]:
data['female'] = data.Lyrics.apply(lambda x: len([y for y in x.split() if y in gender_dict and gender_dict[y] == 'f']))

In [141]:
gender_df = data.groupby('id')[['male','female','words']].sum()

In [153]:
gender_df['male_pct'] = gender_df.apply(lambda x: x['male'] / x['words'] if x['words'] > 0 else 0,axis=1)
gender_df['female_pct'] = gender_df.apply(lambda x: x['female'] / x['words'] if x['words'] > 0 else 0,axis=1)

In [157]:
gender_df.male_pct.describe()

count    1001.000000
mean        0.144830
std         0.234537
min         0.000000
25%         0.000000
50%         0.000000
75%         0.238095
max         1.000000
Name: male_pct, dtype: float64

In [158]:
gender_df.female_pct.describe()

count    1001.000000
mean        0.123185
std         0.233257
min         0.000000
25%         0.000000
50%         0.000000
75%         0.153846
max         1.000000
Name: female_pct, dtype: float64